<a href="https://colab.research.google.com/github/yulk6992/HACKATHON-2/blob/main/hacketon2_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HACKATHON#2 : Système de recherche et de synthèse de documents basés sur l'IA


L'objectif est le suivant:

Construire un outil d'IA permettant d'absorber des documents, faire des recherches sémantiques et générer des résumés sur des sections de manière ciblées.


# 1. Définition du Scope & des types de Documents

* Premièrement, décider quel format de document adopté (PDF, Word, plain text).
* Puis, définir un workflow d'ingestion de haut niveau comme suit:


Téléchargement → Extraction de texte → Intégration → Stockage vectoriel → Interface de requête → Résumé → Sortie

## Enoncé du problème :

Dans de nombreuses organisations, les documents PDF, docs ou TXT contiennent des informations essentielles, mais qui sont souvent difficiles à retrouver rapidement.

Les utilisateurs perdent du temps à parcourir des rapports volumineux, tandis que la recherche par mots-clés reste limitée et imprécise.

Il existe donc un besoin clair d' un système capable de :
- ingérer différents types de documents,
- comprendre leur contenu de manière sémantique,
- retrouver les passages pertinents,
- générer un résumé clair et exploitable.

Notre projet vise à construire un pipeline complet permettant d'aider les utilisateurs à faire des recherches et d'accéder à l'information plus rapidement.


### 1.1 Types de documents supportés

Pour ce prototype, nous choissisons d'utiliser deux formats principaux :
- **PDF (.pdf)**
- **Texte brut (.txt)**

Ces formats sont largement utilisés dans les contextes réels et permettent une ingestion fiable et simple.  
Nous n'incluons pas pour l'instant les fichiers Word (.docx) afin de réduire la complexité, mais il sera facile d'ajouter ce type de fichier en les téléchargeant simplement.  


### 1.2 Workflow d’ingestion

Le pipeline global de traitement suit les étapes suivantes :

Téléchargement → Extraction de texte → Intégration → Stockage vectoriel  (FAISS) → Interface de requête → Résumé → Sortie

Ce workflow servira de fil conducteur pour les sections suivantes.


# 2. Implémentation des documents et Extraction de texte

* La première étape est de configurer le téléchargement de fichiers ou sources par différent biais.  
* Ensuite, nous utiliserons la bibliothèque PyPDF2 pour extraire le texte brut de chaque document.

### 2.1 Interface de téléchargement des documents

Pour l'ingestion des documents, nous mettons en place une interface simple de téléchargement à l'aide de **Streamlit**.  
Ce choix nous permet de :

- créer rapidement une interface web minimale,
- éviter la complexité d'un framework plus lourd comme Flask pour un prototype d'hackathon,
- offrir une intéraction directe et intuitive à l'utilisateur.

L'utilisateur peut charger un fichier **PDF** ou **TXT**, qui est ensuite transmis au module d' extraction du texte.


In [1]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.1 MB/s eta 0:00:00


In [2]:
import streamlit as st

st.title("Document uploader")

uploaded_file = st.file_uploader(
    "Téléchargez un document (PDF ou TXT)",
    type=["pdf", "txt"]
)

if uploaded_file is not None:
    st.success(f"Fichier reçu : {uploaded_file.name}")
    # le fichier sera ensuite passé à la fonction d'extraction de texte (étape 2.2)

2025-11-23 13:58:34.105 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.523 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-23 13:58:34.533 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.552 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.558 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-23 13:58:34.583 Thread 'MainThread': mi

### 2.2 Extraction du texte

Une fois le fichier téléchargé, nous devons extraire le texte brut afin de pouvoir le segmenter puis le vectoriser.

Nous utilisons :
- **PyPDF2** pour lire le contenu des fichiers **PDF**,
- une simple lecture (`read()`) pour les fichiers **TXT**.

L'objectif est d'obtenir une chaîne de caractères unique (`extracted_text`) qui servira de point de départ pour l'étape 3 (Split & Préprocessing du Texte).


In [3]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [4]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file):  # extraction depuis un PDF (avec PyPDF2)
    reader = PdfReader(file)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

def extract_text_from_txt(file):    # extraction depuis un fichier TXT
    return file.read().decode("utf-8")

def extract_text(uploaded_file):   # routage automatique selon le type de fichier
    filename = uploaded_file.name.lower()
    if filename.endswith(".pdf"):
        return extract_text_from_pdf(uploaded_file)
    elif filename.endswith(".txt"):
        return extract_text_from_txt(uploaded_file)
    else:
        raise ValueError("Format non supporté (utiliser .pdf ou .txt)")

# Exemple d'utilisation (dans Streamlit) :
# if uploaded_file is not None:
#     extracted_text = extract_text(uploaded_file)
#     st.text(extracted_text[:500])

Import des fichiers via le "folder" d'un utilisateur

In [5]:
from google.colab import files

print("Please select fewer files for upload.")
uploaded = files.upload()
print("\nFile loaded:", list(uploaded.keys()))

Please select fewer files for upload.


Saving open_science_summary_1.txt to open_science_summary_1.txt
Saving preprocessing_guide.txt to preprocessing_guide.txt
Saving semantic_search_intro.txt to semantic_search_intro.txt
Saving teacher_shortages_notes.txt to teacher_shortages_notes.txt
Saving teachers_summary_1.txt to teachers_summary_1.txt

File loaded: ['open_science_summary_1.txt', 'preprocessing_guide.txt', 'semantic_search_intro.txt', 'teacher_shortages_notes.txt', 'teachers_summary_1.txt']


Import de fichiers "pdf" via un URL source et téléchargement (uniquement pour faire le test du code en end to end)

In [6]:
URLS = ('https://www.ouvrirlascience.fr/wp-content/uploads/2021/11/UNESCO_Recommendation-on-Open-Science_november2021.pdf',
        "https://teachertaskforce.org/sites/default/files/2024-02/2024_TTF-UNESCO-Global-Report-on-Teachers_EN.pdf"
        )

In [7]:
import os

for url in URLS:
    # Extract filename from URL
    filename = url.split('/')[-1] # This gets the last part of the URL path

    # Use wget to download the file, saving it with the extracted filename
    !wget -q -O "{filename}" "{url}"
    print(f"Downloaded: {filename}")

# Verify that the files are downloaded
print("\nFiles in current directory:")
!ls *.pdf

Downloaded: UNESCO_Recommendation-on-Open-Science_november2021.pdf
Downloaded: 2024_TTF-UNESCO-Global-Report-on-Teachers_EN.pdf

Files in current directory:
2024_TTF-UNESCO-Global-Report-on-Teachers_EN.pdf
UNESCO_Recommendation-on-Open-Science_november2021.pdf


In [8]:
from PyPDF2 import PdfReader
import io

# Fonction pour extraire le texte depuis un PDF
# Utilise PyPDF2 pour lire chaque page du document
def extract_text_from_pdf(file_bytes):
    reader = PdfReader(io.BytesIO(file_bytes))
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"    # ajouter le texte de chaque page
    return text

# Fonction pour extraire le texte depuis un fichier .txt
# Décodage simple du contenu en UTF-8
def extract_text_from_txt(file_bytes):
    return file_bytes.decode("utf-8")

# Dictionnaire pour stocker le texte extrait de chaque fichier
all_documents_text = {}

# Parcours de tous les fichiers téléchargés via files.upload()
for filename, file_bytes in uploaded.items():
    name = filename.lower()

    # Détection du type de fichier (PDF ou TXT)
    if name.endswith(".pdf"):
        text = extract_text_from_pdf(file_bytes)
    elif name.endswith(".txt"):
        text = extract_text_from_txt(file_bytes)
    else:
        print(f"Format non supporté : {filename}")
        continue

    # On ajoute le texte extrait dans le dictionnaire
    all_documents_text[filename] = text

    # Affichage des premiers caractères pour vérification
    print(f"\n--- {filename} ---")
    print(text[:300])



--- open_science_summary_1.txt ---
UNESCO Recommendation on Open Science – Summary

Open Science promotes transparency, accessibility, and collaboration in research.
The document encourages open access publishing, data sharing, and global scientific cooperation.
Member states are urged to adopt national policies that remove barri

--- preprocessing_guide.txt ---
How to Preprocess Text for Embeddings

Clean the text by removing extra spaces, tabs, and repeated lines.
Split the text into small chunks of 200–500 tokens.
Normalize the text and remove irrelevant formatting.
These steps improve both embedding quality and retrieval accuracy.


--- semantic_search_intro.txt ---
Introduction to Semantic Search

Semantic search focuses on meaning rather than keywords.
Instead of matching exact words, it uses vector embeddings to represent concepts.
This allows retrieval of relevant passages even when different wording is used.


--- teacher_shortages_notes.txt ---
Notes on Teacher Shortages

- 

# 3. Split & Preprocessing du Texte

* Tout d'abord, on segmente le texte extrait en morceaux gérables "chunk" (par exemple, 200 à 500 tokens par morceau).
* Ensuite, on nettoie et normalise chacun des morceaux (suppriression des en-têtes, des pieds de page et des espaces).

### 3.1 Segmentation du texte

Après l'extraction, le texte obtenu peut être très volumineux.  
Pour permettre une recherche sémantique efficace, nous devons le découper en **petits segments** ("chunks") de taille gérable.

Nous choisissons une taille d'environ **200 - 500 tokens** (ou ~600-900 caractères).  

Cela permet :

- d'améliorer la qualité des "embeddings",
- de rendre la recherche plus précise dans la base vectorielle,
- d'éviter de dépasser les limites de contexte des modèles de résumé,
- d'assurer une navigation plus fine dans les documents.

Chaque segment devient ainsi une unité indépendante prête pour les étapes suivantes :  
**nettoyage → embeddings → indexation → recherche → résumé**.


In [9]:
# Fonction pour segmenter un texte long en petits blocs (chunks)
# On coupe le texte selon les paragraphes, sans dépasser une longueur maximale

def split_text_into_chunks(text, max_chars=900):
    paragraphs = text.split("\n\n")   # séparation par paragraphes
    chunks = []
    current_chunk = ""

    for p in paragraphs:
        p = p.strip()
        if not p:
            continue  # ignorer les lignes vides

        # Si le paragraphe tient dans le chunk actuel → on l'ajoute
        if len(current_chunk) + len(p) < max_chars:
            current_chunk += p + "\n\n"
        else:
            # Sinon, on ferme le chunk actuel et on en crée un nouveau
            chunks.append(current_chunk.strip())
            current_chunk = p + "\n\n"

    # Ajouter le dernier chunk s'il n'est pas vide
    if current_chunk.strip():
        chunks.append(current_chunk.strip())

    return chunks

# Application de la segmentation à tous les documents extraits
all_documents_chunks = {}

for filename, text in all_documents_text.items():
    chunks = split_text_into_chunks(text)
    all_documents_chunks[filename] = chunks
    print(f"{filename} → {len(chunks)} chunks créés")
    print("Exemple de chunk :\n", chunks[0][:350], "\n---\n")


open_science_summary_1.txt → 1 chunks créés
Exemple de chunk :
 UNESCO Recommendation on Open Science – Summary

Open Science promotes transparency, accessibility, and collaboration in research.
The document encourages open access publishing, data sharing, and global scientific cooperation.
Member states are urged to adopt national policies that remove barriers to scientific knowledge. 
---

preprocessing_guide.txt → 1 chunks créés
Exemple de chunk :
 How to Preprocess Text for Embeddings

Clean the text by removing extra spaces, tabs, and repeated lines.
Split the text into small chunks of 200–500 tokens.
Normalize the text and remove irrelevant formatting.
These steps improve both embedding quality and retrieval accuracy. 
---

semantic_search_intro.txt → 1 chunks créés
Exemple de chunk :
 Introduction to Semantic Search

Semantic search focuses on meaning rather than keywords.
Instead of matching exact words, it uses vector embeddings to represent concepts.
This allows retrieval of 

### 3.2 Pré-traitement et normalisation du texte

Avant de générer des embeddings, nous appliquons un pré-traitement léger sur chaque chunk.

Objectifs du pré-traitement :
- supprimer les espaces inutiles (début/fin de texte),
- enlever les lignes vides,
- normaliser les sauts de ligne,
- réduire le bruit éventuel lié à la mise en page.

Cela permet d'obtenir des segments plus propres, ce qui améliore la qualité des embeddings et la pertinence de la recherche sémantique.


In [10]:
# Fonction de nettoyage d'un chunk de texte
# - suppression des espaces au début/à la fin
# - suppression des lignes vides
# - normalisation des tabulations

def clean_chunk(chunk):
    # retirer espaces au début/à la fin
    cleaned = chunk.strip()
    # remplacer les tabulations par un espace
    cleaned = cleaned.replace("\t", " ")
    # supprimer les lignes vides et nettoyer chaque ligne
    cleaned_lines = []
    for line in cleaned.splitlines():
        line = line.strip()
        if line:  # garder seulement les lignes non vides
            cleaned_lines.append(line)
    # reconstruire le texte
    cleaned = "\n".join(cleaned_lines)
    return cleaned

# Application du pré-traitement à tous les chunks de tous les documents
all_documents_clean_chunks = {}

for filename, chunks in all_documents_chunks.items():
    clean_list = [clean_chunk(c) for c in chunks]
    all_documents_clean_chunks[filename] = clean_list

    print(f"\n=== {filename} ===")
    print(f"Nombre de chunks nettoyés : {len(clean_list)}")
    print("Exemple de chunk nettoyé :\n", clean_list[0][:350], "\n---")



=== open_science_summary_1.txt ===
Nombre de chunks nettoyés : 1
Exemple de chunk nettoyé :
 UNESCO Recommendation on Open Science – Summary
Open Science promotes transparency, accessibility, and collaboration in research.
The document encourages open access publishing, data sharing, and global scientific cooperation.
Member states are urged to adopt national policies that remove barriers to scientific knowledge. 
---

=== preprocessing_guide.txt ===
Nombre de chunks nettoyés : 1
Exemple de chunk nettoyé :
 How to Preprocess Text for Embeddings
Clean the text by removing extra spaces, tabs, and repeated lines.
Split the text into small chunks of 200–500 tokens.
Normalize the text and remove irrelevant formatting.
These steps improve both embedding quality and retrieval accuracy. 
---

=== semantic_search_intro.txt ===
Nombre de chunks nettoyés : 1
Exemple de chunk nettoyé :
 Introduction to Semantic Search
Semantic search focuses on meaning rather than keywords.
Instead of matching ex

# 4. Generation des "Embeddings"

* Après le chargement d'un modèle de transformation de phrases (par exemple, all-MiniLM-L6-v2), nous calculons les "embeddings" pour chaque bloc de texte.

In [11]:
from sentence_transformers import SentenceTransformer

# utilisation du modèle "all-MiniLM-L6-V2"
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Populate data with all cleaned chunks from all_documents_clean_chunks
data = []
for filename, chunks in all_documents_clean_chunks.items():
    data.extend(chunks)

embeddings = model.encode(data)

# Only print dimensions if embeddings are not empty to avoid IndexError
if len(embeddings) > 0:
    print(f"Number of embeddings: {len(embeddings)}, Dimension of embeddings: {len(embeddings[0])}")
else:
    print("No embeddings were generated because the data list was empty.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of embeddings: 5, Dimension of embeddings: 384


In [12]:
import numpy as np

# Save the embeddings to a .npy file
np.save('embeddings.npy', embeddings)
print('Embeddings saved to embeddings.npy')

Embeddings saved to embeddings.npy


# 5. Construction et remplissage de la base vectorielle

* On Commence par initialiser l'index FAISS avec des paramètres adaptés au CPU.

* Ensuite, on insère nos "chunk embeddings" et on conserve les métadonnées (ID du document, index du segment).

In [13]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 58.5 MB/s eta 0:00:00


Creation de l'index FAISS

In [14]:
import faiss
import numpy as np

# dimension = taille d'un embedding
dim = embeddings.shape[1]

# Index le plus simple, le plus fiable, le plus CPU-friendly
index = faiss.IndexFlatL2(dim)

# Ajouter les embeddings à l'index
index.add(embeddings.astype(np.float32))

print("Nombre de vecteurs dans l'index =", index.ntotal)

Nombre de vecteurs dans l'index = 5


# 6. Créer l'interface de recherche

* On commence par créer un formulaire de saisie de requête dans notre application.
* Ensuite, lors de la soumission de la requête, on réalise les étapes : intégrer la requête, rechercher les segments top-k dans le magasin vectoriel et renvoyer les plus pertinents.

Recherche sémantique (Top-k)

In [15]:
def search(query, k=5):
    # Encoder la requête
    q_emb = model.encode([query], convert_to_numpy=True)

    # Lancer la recherche dans FAISS
    distances, indices = index.search(q_emb.astype(np.float32), k)

    return distances[0], indices[0]

In [16]:
query = "explain open science"
distances, indices = search(query, k=5)

indices, distances

(array([0, 3, 2, 4, 1]),
 array([0.8060622, 1.8040802, 1.8110157, 1.8234961, 1.9647087],
       dtype=float32))

Récupération du texte

In [17]:
metadata = []
# Iterate through all cleaned chunks from all documents to build the metadata list
for filename, chunks in all_documents_clean_chunks.items():
    for i, chunk_text in enumerate(chunks):
        metadata.append({
            "source": filename,
            "chunk_id": i,
            "text": chunk_text
        })

def get_results(query, k=5):
    # Encoder la requête
    q_emb = model.encode([query], convert_to_numpy=True)

    # Lancer la recherche dans FAISS
    distances, indices = index.search(q_emb.astype(np.float32), k)

    results = []

    for d, idx in zip(distances[0], indices[0]):
        meta = metadata[idx] # metadata is now defined
        results.append({
            "distance": float(d),
            "source": meta["source"],
            "chunk_id": meta["chunk_id"],
            "text": meta["text"]
        })
    return results


Affichage du résultat

In [18]:
results = get_results("summary the information of open science", k=5)

for i, r in enumerate(results):
    print(f"\n--- Résultat {i+1} ---")
    print("Source :", r["source"])
    print("Chunk :", r["chunk_id"])
    print("Distance :", round(r["distance"], 4))
    print("\nTexte :")
    print(r["text"][:500], "...")  # tronque pour lisibilité


--- Résultat 1 ---
Source : open_science_summary_1.txt
Chunk : 0
Distance : 0.6401

Texte :
UNESCO Recommendation on Open Science – Summary
Open Science promotes transparency, accessibility, and collaboration in research.
The document encourages open access publishing, data sharing, and global scientific cooperation.
Member states are urged to adopt national policies that remove barriers to scientific knowledge. ...

--- Résultat 2 ---
Source : teachers_summary_1.txt
Chunk : 0
Distance : 1.693

Texte :
UNESCO Global Report on Teachers (2024) – Summary
The report highlights the global teacher shortage, especially in low-income countries.
Key issues include lack of training, poor working conditions, and insufficient career development.
UNESCO recommends increasing investment in teacher education and improving professional support. ...

--- Résultat 3 ---
Source : semantic_search_intro.txt
Chunk : 0
Distance : 1.7083

Texte :
Introduction to Semantic Search
Semantic search focuses on mea

7. Résumer le contenu récupéré

* Premièrement, on charge un modèle de résumé (par exemple, t5-small).
* Ensuite, on transmet les segments top-k concaténés au résumeur afin d'obtenir un résumé concis.

In [19]:
!pip install transformers sentencepiece

In [20]:
from transformers import pipeline, AutoTokenizer

summarizer = pipeline(
    "summarization",
    model="t5-small",
    tokenizer="t5-small",
    device=-1
)
tokenizer = AutoTokenizer.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [21]:
retrieved_text = " ".join([r["text"] for r in results])
print("Lenght text retrived:", len(retrieved_text))

Lenght text retrived: 1445


In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
summary = summarizer(
    "summarize: " + retrieved_text,
    max_new_tokens=120,   # instade of max_length → more stabil
    do_sample=False,
    truncation=True
)[0]["summary_text"]
print("\n=== SUMMARY ===\n", summary)


=== SUMMARY ===
 the report highlights the global teacher shortage . key issues include lack of training, poor working conditions, and insufficient career development . UNESCO recommends increasing investment in teacher education and improving professional support .


In [24]:
def summarize_text(text, min_length=30, max_length=150):
    tokens = tokenizer.encode(text)
    if len(tokens) > tokenizer.model_max_length:
        tokens = tokens[:tokenizer.model_max_length]
        text = tokenizer.decode(tokens)

    summary = summarizer(
        text,
        min_length=min_length,
        max_length=max_length,
        do_sample=False
    )[0]["summary_text"]

    return summary

# 8. Évaluer la recherche et la synthèse

* La première étape consiste à constituer un ensemble de requêtes test avec des passages et des résumés pertinents servant de référence.
* Ensuite, on évalue le résultat :

        * Recherche : précision@k, rappel@k
        * Synthèse : BLEU, ROUGE, perplexité non vu et remplacé par Input d'utilisateur

### 8-1 Création d'une requête et évaluation du résultat

In [31]:
# === Test set for evaluation (query, ground-truth text, expected summary) ===
# Updated ground_truth texts to be actual snippets from the loaded documents
test_set = [
    {
        "query": "What is open science about?",
        "ground_truth": "Open Science promotes transparency, accessibility, and collaboration in research.",
        "expected_summary": "Open science promotes transparency, accessibility, and collaboration."
    },
    {
        "query": "Tell me about the global teacher shortage.",
        "ground_truth": "The report highlights the global teacher shortage, especially in low-income countries.",
        "expected_summary": "The global teacher shortage is particularly acute in low-income countries."
    },
    {
        "query": "What is semantic search?",
        "ground_truth": "Semantic search focuses on meaning rather than keywords.",
        "expected_summary": "Semantic search uses vector embeddings to understand meaning, not just keywords."
    },
]

In [36]:
#SEARCH EVALUATION (precision@k et recall@k)
def evaluate_search(test_set, k=5):
    precision_list = []
    recall_list = []
    for test in test_set:
        query = test["query"]
        ground_truth_text = test["ground_truth"]
        results = get_results(query, k)
        retrieved_texts = [r["text"] for r in results]
        # Match if the real text shows up partially in RAG results
        matches = sum(1 for t in retrieved_texts if ground_truth_text[:50].lower() in t.lower())
        precision = matches / k
        recall = matches / 1   # each query has just one match
        precision_list.append(precision)
        recall_list.append(recall)
    return precision_list, recall_list

In [34]:
precision_list, recall_list = evaluate_search(test_set)
print(f"Precision@k: {precision_list}")
print(f"Recall@k: {recall_list}")
print(f"Average precision: {sum(precision_list)/len(precision_list):.4f}")
print(f"Average recall: {sum(recall_list)/len(recall_list):.4f}")

Precision@k: [0.2, 0.2, 0.2]
Recall@k: [1.0, 1.0, 1.0]
Average precision: 0.2000
Average recall: 1.0000


### 8.2 - Evaluation du Résumé

In [39]:
# SUMMARIZATION EVALUATION
print("Please provide your human evaluation scores (1-5):")

# Get user input for clarity, relevance, and fluency
clarity = float(input("Clarity (1-5): "))
relevance = float(input("Relevance (1-5): "))
fluency = float(input("Fluency (1-5): "))

# Calculate the average score
average_score = (clarity + relevance + fluency) / 3

print(f"\nYour scores: Clarity={clarity}, Relevance={relevance}, Fluency={fluency}")
print(f"Average Evaluation Score: {average_score:.2f}")

Please provide your human evaluation scores (1-5):
Clarity (1-5): 4
Relevance (1-5): 3
Fluency (1-5): 3

Your scores: Clarity=4.0, Relevance=3.0, Fluency=3.0
Average Evaluation Score: 3.33


In [40]:
# Coherence Evaluation replace perplexity
print("""
=== Coherence Evaluation ===
- Does the generated summary look like it was written by a human?
- Are there contradictions?
- Are the sentences complete and logical?
- Is the tone consistent?
Write your observations below:
""")


=== Coherence Evaluation ===
- Does the generated summary look like it was written by a human?
- Are there contradictions?
- Are the sentences complete and logical?
- Is the tone consistent?
Write your observations below:



# 9. Optimisation pour CPU & plus grande échelle


Pour cette étape nous allons rendre le pipeline plus robuste sur CPU en :

* encodant et résumant par micro-lots (batch size 2–4),
* conservant un index FAISS léger (`IndexFlat*`) normalisé pour le cosinus,
* limitant le nombre de chunks maintenus en mémoire grâce à un « rolling store ».

Les cellules suivantes ajoutent les utilitaires nécessaires puis reconstruisent l’index optimisé.

### 9.1 Helpers pour batching & gestion de chunks

Le bloc suivant reprend les utilitaires nécessaires (équivalent du module `hackathon2_step9.py`) afin que le notebook reste autonome.


In [41]:
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Iterable, List, Sequence, Tuple

import faiss
import numpy as np


def batch_encode_texts(
    model,
    chunks: Sequence[str],
    batch_size: int = 4,
    show_progress_bar: bool = True,
) -> np.ndarray:
    """Encode les chunks par micro-lots (2-4) pour rester compatible CPU."""
    if not chunks:
        dim = model.get_sentence_embedding_dimension()
        return np.empty((0, dim), dtype="float32")

    embeddings: List[np.ndarray] = []
    for start in range(0, len(chunks), batch_size):
        batch = list(chunks[start : start + batch_size])
        batch_embeddings = model.encode(
            batch,
            convert_to_numpy=True,
            show_progress_bar=show_progress_bar if start == 0 else False,
        )
        embeddings.append(batch_embeddings)

    return np.vstack(embeddings).astype("float32")


def summarize_in_batches(
    summarizer_pipeline,
    texts: Sequence[str],
    batch_size: int = 3,
    instruction: str = "summarize: ",
) -> List[str]:
    """Concatène de petits lots de textes avant de les envoyer au modèle de résumé."""
    summaries: List[str] = []
    for start in range(0, len(texts), batch_size):
        batch = texts[start : start + batch_size]
        if not batch:
            continue
        prompt = instruction + " ".join(batch)
        output = summarizer_pipeline(
            prompt,
            max_new_tokens=180,
            do_sample=False,
            truncation=True,
        )[0]["summary_text"]
        summaries.append(output)
    return summaries


def build_faiss_index(embeddings: np.ndarray, use_cosine_similarity: bool = True) -> faiss.Index:
    """Construit un index FAISS léger (IndexFlatIP ou IndexFlatL2)."""
    if embeddings.dtype != np.float32:
        embeddings = embeddings.astype("float32")

    if embeddings.size:
        dim = embeddings.shape[1]
    else:
        dim = model.get_sentence_embedding_dimension()

    if use_cosine_similarity:
        faiss.normalize_L2(embeddings)
        index = faiss.IndexFlatIP(dim)
    else:
        index = faiss.IndexFlatL2(dim)

    if embeddings.size:
        index.add(embeddings)
    return index


@dataclass
class ChunkMetadata:
    source: str
    chunk_id: int
    text: str


@dataclass
class RollingChunkStore:
    """Maintient un quota de chunks actifs et évince les plus anciens si besoin."""

    max_chunks: int = 2000
    texts: List[str] = field(default_factory=list)
    metadata: List[ChunkMetadata] = field(default_factory=list)

    def add_document(self, filename: str, chunks: Iterable[str]) -> None:
        for idx, chunk in enumerate(chunks):
            self.texts.append(chunk)
            self.metadata.append(ChunkMetadata(filename, idx, chunk))
            self._evict_if_needed()

    def _evict_if_needed(self) -> None:
        overflow = len(self.texts) - self.max_chunks
        if overflow <= 0:
            return
        del self.texts[:overflow]
        del self.metadata[:overflow]

    def export(self) -> Tuple[List[str], List[ChunkMetadata]]:
        return list(self.texts), list(self.metadata)


def rebuild_index_from_store(
    store: RollingChunkStore,
    encoder_model,
    batch_size: int = 4,
) -> Tuple[faiss.Index, np.ndarray, List[ChunkMetadata]]:
    """Ré-encode tous les chunks actifs et retourne l'index FAISS optimisé."""
    texts, metadata = store.export()
    embeddings = batch_encode_texts(encoder_model, texts, batch_size=batch_size)
    index = build_faiss_index(embeddings, use_cosine_similarity=True)
    return index, embeddings, metadata

### 9.2 Reconstruction de l’index optimisé

On reconstruit maintenant l’index FAISS et les métadonnées en s’appuyant sur le `RollingChunkStore` (quota 2 000 chunks) et sur le batching CPU.


In [42]:
store = RollingChunkStore(max_chunks=2000)
for filename, chunks in all_documents_clean_chunks.items():
    store.add_document(filename, chunks)

index, embeddings, metadata_objects = rebuild_index_from_store(
    store,
    model,
    batch_size=4,
)

metadata = [
    {"source": meta.source, "chunk_id": meta.chunk_id, "text": meta.text}
    for meta in metadata_objects
]
np.save("embeddings.npy", embeddings)
print(
    f"Index optimisé reconstruit avec {index.ntotal} vecteurs (quota chunks={store.max_chunks})."
)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Index optimisé reconstruit avec 5 vecteurs (quota chunks=2000).


# 10. (Bonus) Automatisation du Processus pour de nouveau document

## 10.1 Automatisation via application Streamlit

Pour automatiser le traitement des nouveaux documents, nous avons créé une application Streamlit (`app.py`) qui :

1. **Détecte automatiquement les nouveaux uploads** : Dès qu'un fichier est déposé via l'interface, le pipeline complet se déclenche automatiquement.

2. **Traite immédiatement** : Extraction → Segmentation → Embeddings → Indexation FAISS se fait sans intervention manuelle.

3. **Gère la session** : Les documents restent disponibles pour la recherche jusqu'à ce que la session soit réinitialisée.

### Avantages de cette approche

- **Pas de file d'attente complexe** : Streamlit gère automatiquement les événements d'upload
- **Interface intuitive** : L'utilisateur voit directement le traitement en cours
- **Déploiement simple** : Compatible avec Streamlit Cloud pour un accès en ligne
- **Traitement en temps réel** : Les documents sont immédiatement disponibles pour la recherche


## 10.2 Utilisation de l'application Streamlit

### Installation et lancement local

```bash
# Installer les dépendances
pip install streamlit faiss-cpu sentence-transformers transformers PyPDF2

# Lancer l'application
streamlit run app.py -> si vous êtes dans le répertoire de l'application, sinon il faudra mettre le chemin complet.
streamlit run "CHEMIN DU FICHIER APP.PY"
```

L'application sera accessible sur `http://localhost:8501`

### Déploiement sur Streamlit Cloud -> Notre choix s'est porté sur cette méthode

1. **Créer un compte** sur [share.streamlit.io](https://share.streamlit.io)
2. **Connecter le dépôt GitHub** contenant `app.py` et `requirements.txt`
3. **Déployer** : Streamlit Cloud détecte automatiquement le fichier et installe les dépendances
4. **Accès en ligne** : L'application est accessible via une URL publique -> https://hackathon2-rag.streamlit.app/

### Fonctionnalités de l'app

- **Upload multiple** : Possibilité de charger plusieurs fichiers PDF/TXT en une fois
- **Traitement automatique** : Chaque fichier est traité dès l'upload
- **Recherche sémantique** : Interface de recherche avec affichage des top-k résultats
- **Résumé automatique** : Génération de résumés pour les textes > 200 caractères
- **Réinitialisation** : Bouton pour vider le cache et recommencer


## 10.3 Code clé de l'automatisation dans app.py

Le mécanisme d'automatisation repose sur la détection automatique des uploads dans Streamlit :

```python
# Dans app.py, section upload
uploaded_files = st.file_uploader(
    "Déposez vos PDF/TXT (traitement immédiat)",
    type=["pdf", "txt"],
    accept_multiple_files=True,
)

if uploaded_files:  # Déclenchement automatique dès qu'un fichier est détecté
    for uploaded_file in uploaded_files:
        with st.spinner(f"Ingestion de {uploaded_file.name}..."):
            # Pipeline complet exécuté automatiquement
            raw_text = extract_text(uploaded_file)
            chunks = split_text_into_chunks(raw_text)
            clean_chunks = [clean_chunk(c) for c in chunks]
            
            # Ajout au store et reconstruction de l'index
            st.session_state.store.add_document(uploaded_file.name, clean_chunks)
            index, embeddings, metadata = rebuild_index_from_store(
                st.session_state.store,
                encoder_model,
                batch_size=4,
            )
            st.session_state.faiss_index = index
            st.session_state.metadata = metadata
        st.success(f"{uploaded_file.name} ingéré ({len(clean_chunks)} chunks)")
```

### Points importants

- **Pas de bouton "Traiter"** : Le traitement se fait automatiquement à l'upload
- **Session persistante** : Les documents restent en mémoire pendant la session
- **Gestion des erreurs** : Les erreurs sont affichées directement dans l'interface
- **Feedback visuel** : Spinner et messages de succès pour informer l'utilisateur

### Alternative : File watcher (pour usage avancé)

Pour une automatisation plus avancée (surveillance d'un dossier), on pourrait utiliser `watchdog` :

```python
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class DocumentHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.src_path.endswith(('.pdf', '.txt')):
            # Traiter le nouveau fichier
            process_document(event.src_path)

observer = Observer()
observer.schedule(DocumentHandler(), path='./documents/', recursive=False)
observer.start()
```

Cette approche est plus complexe et nécessite un serveur dédié, ce qui dépasse le scope de ce hackathon.


# 11. Documentation & Réflection


## 11.1 Résumé de l'architecture système

### Architecture globale

Notre système suit un pipeline RAG (Retrieval-Augmented Generation) composé des étapes suivantes :

```
Upload → Extraction texte → Segmentation → Nettoyage → Embeddings → Index FAISS → Recherche → Résumé
```

### Composants principaux

1. **Interface utilisateur** : Streamlit
   - Choix : Framework simple et rapide pour prototyper
   - Avantages : Interface web native, widgets intégrés, déploiement facile

2. **Extraction de texte** : PyPDF2
   - Choix : Bibliothèque légère et fiable pour PDF
   - Alternative considérée : PyMuPDF (plus rapide mais plus complexe)

3. **Segmentation** : Algorithme personnalisé par paragraphes
   - Taille des chunks : 200-500 tokens (environ 600-900 caractères)
   - Justification : Équilibre entre granularité et contexte

4. **Embeddings** : Sentence Transformers (all-MiniLM-L6-v2)
   - Choix : Modèle compact (80MB), rapide, qualité correcte
   - Dimension : 384
   - Alternative : all-mpnet-base-v2 (meilleure qualité mais plus lent)

5. **Base vectorielle** : FAISS (IndexFlatIP)
   - Choix : Index simple, CPU-friendly, pas de GPU requis
   - Métrique : Similarité cosinus (produit scalaire sur vecteurs normalisés)
   - Alternative évitée : IVF, PQ (trop complexes pour ce cas d'usage)

6. **Résumé** : T5-small
   - Choix : Modèle léger, rapide, bon compromis qualité/vitesse
   - Alternative : BART-base (meilleure qualité mais plus lourd)


## 11.2 Analyse de la précision des recherches

### Métriques évaluées

**Precision@k** : Proportion de résultats pertinents parmi les k premiers résultats retournés.

**Recall@k** : Proportion de documents pertinents retrouvés parmi les k premiers résultats.

### Observations

- **Recherche sémantique efficace** : Le système trouve des passages pertinents même avec des requêtes qui ne correspondent pas exactement aux mots du document (ex: "embddeing" trouve "embeddings").

- **Scores de similarité** : Les scores varient généralement entre 0.1 et 0.9 selon la pertinence. Un score > 0.5 indique généralement une bonne correspondance.

- **Limitations** :
  - Les documents très courts (1 seul chunk) peuvent retourner des résultats dupliqués (corrigé avec déduplication)
  - La qualité dépend fortement de la qualité des embeddings et de la segmentation

### Améliorations possibles

- Utiliser un modèle d'embedding plus performant (all-mpnet-base-v2)
- Ajuster la taille des chunks selon le type de document
- Ajouter un re-ranking avec un modèle cross-encoder


## 11.3 Analyse de la qualité de synthèse

### Évaluation qualitative

**Clarté (1-5)** : Le résumé est-il clair et facile à comprendre ?
- Observation : Les résumés générés sont généralement cohérents mais peuvent être répétitifs pour des textes courts.

**Pertinence (1-5)** : Le résumé répond-il à la question posée ?
- Observation : Le résumé capture les points clés mais peut manquer de contexte spécifique à la requête.

**Fluidité (1-5)** : Le texte est-il naturel et fluide ?
- Observation : T5-small produit des résumés grammaticalement corrects mais parfois mécaniques.

### Stratégie de résumé conditionnel

Nous avons implémenté une logique qui génère un résumé uniquement si le texte total dépasse 200 caractères. Pour les textes courts, les résultats de recherche constituent déjà un résumé efficace.

### Améliorations possibles

- Utiliser un modèle de résumé plus performant (BART-large, T5-base)
- Implémenter un résumé guidé par la requête (query-focused summarization)
- Ajouter un post-traitement pour améliorer la fluidité


## 11.4 Analyse des compromis de performance CPU

### Optimisations implémentées

1. **Batching des embeddings** (batch_size=4)
   - Impact : Réduit la charge mémoire et améliore le throughput
   - Compromis : Légèrement plus lent qu'un encodage massif mais plus stable

2. **Batching des résumés** (batch_size=3)
   - Impact : Évite les timeouts et les débordements mémoire
   - Compromis : Résumés légèrement moins cohérents mais génération plus fiable

3. **Index FAISS simplifié** (IndexFlatIP)
   - Impact : Pas de GPU requis, fonctionnement sur CPU standard
   - Compromis : Recherche légèrement plus lente que les index avancés (IVF, PQ) mais suffisante pour < 10k documents

4. **Gestion des chunks** (RollingChunkStore, max_chunks=2000)
   - Impact : Limite la mémoire utilisée et évite les ralentissements
   - Compromis : Les documents les plus anciens sont évincés automatiquement

### Performances observées

- **Temps d'ingestion** : ~2-5 secondes par document PDF moyen (10-20 pages)
- **Temps de recherche** : < 100ms pour une requête
- **Temps de résumé** : 1-3 secondes selon la longueur du texte

### Compromis acceptés

- **Qualité vs Vitesse** : Choix de modèles légers (all-MiniLM-L6-v2, T5-small) pour privilégier la vitesse
- **Précision vs Simplicité** : Index FAISS simple plutôt qu'index avancés pour faciliter le déploiement
- **Mémoire vs Capacité** : Limite de 2000 chunks pour rester compatible avec des environnements limités

### Recommandations pour la production

- Utiliser un GPU pour les embeddings et résumés si le volume est important
- Implémenter un cache pour les requêtes fréquentes
- Utiliser un index FAISS plus avancé (IVF) si > 50k documents
- Ajouter une base de données persistante pour les métadonnées


## 11.5 Conclusion et perspectives

### Points forts du système

* **Pipeline complet fonctionnel** : De l'ingestion à la génération de résumés  
* **Optimisé pour CPU** : Fonctionne sans GPU, accessible sur des machines standard  
* **Interface intuitive** : Streamlit permet une utilisation simple et directe  
* **Recherche sémantique efficace** : Trouve des résultats pertinents même avec des requêtes approximatives  
* **Déploiement facile** : Compatible avec Streamlit Cloud et déploiements locaux

### Limitations identifiées

* **Qualité des résumés** : Modèles légers (T5-small) produisent des résumés parfois mécaniques  
* **Scalabilité** : Index FAISS simple limite à ~10k documents pour des performances optimales  
* **Formats supportés** : Seulement PDF et TXT (pas de Word, Excel, etc.)  
* **Pas de persistance** : Les données sont perdues au redémarrage (sauf si sauvegardées manuellement)

### Pistes d'amélioration futures

◼ **Court terme** :
- Ajouter le support des fichiers Word (.docx)
- Implémenter une sauvegarde automatique de l'index FAISS
- Améliorer l'interface avec des filtres par document

◼ **Moyen terme** :
- Intégrer un modèle de re-ranking pour améliorer la précision
- Ajouter un système de cache pour les requêtes fréquentes
- Implémenter une évaluation automatique (ROUGE, BLEU)

◼ **Long terme** :
- Support multi-langues
- Interface de gestion des documents (suppression, mise à jour)
- Déploiement avec base de données persistante (PostgreSQL + FAISS)
- Intégration avec des APIs externes (Google Drive, Dropbox)

---

**Fin du Hackathon #2 - Système de recherche et de synthèse de documents basés sur l'IA**
